In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import random
import os

from scipy.signal import butter, lfilter
from scipy.stats import skew, kurtosis
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.neighbors import NearestCentroid

In [2]:
seed = 57
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)

In [3]:
x = pickle.load(open('x.pkl', 'rb'))
y = pickle.load(open('y.pkl', 'rb'))

In [4]:
x_normal = np.concatenate((x[:300], x[400:]), axis=0)
x_seizure = x[300:400]
print(x_normal.shape)
print(x_seizure.shape)
sampling_freq = 173.6 #based on info from website

(400, 4097)
(100, 4097)


In [5]:
b, a = butter(3, [0.5,40], btype='bandpass',fs=sampling_freq)

x_normal_filtered = np.array([lfilter(b,a,x_normal[ind,:]) for ind in range(x_normal.shape[0])])
x_seizure_filtered = np.array([lfilter(b,a,x_seizure[ind,:]) for ind in range(x_seizure.shape[0])])
print(x_normal.shape)
print(x_seizure.shape)

x_normal = x_normal_filtered
x_seizure = x_seizure_filtered

x = np.concatenate((x_normal,x_seizure))
y = np.concatenate((np.zeros((400,1)),np.ones((100,1))))

(400, 4097)
(100, 4097)


In [6]:
arr = np.zeros((500,61))

In [7]:
#mean
for i in range(500):
    arr[i,0] = np.mean(x[i])

In [8]:
#std
for i in range(500):
    arr[i,1] = np.std(x[i])

In [9]:
#max
for i in range(500):
    arr[i,2] = max(x[i])

In [10]:
#min
for i in range(500):
    arr[i,3] = min(x[i])

In [11]:
#skewness
for i in range(500):
    arr[i,4] = skew(x[i], axis=0, bias=True)

In [12]:
#kurtosis
for i in range(500):
    arr[i,5] = kurtosis(x[i], fisher=True)

In [13]:
#ppt
for i in range(500):
    arr[i,6] = np.ptp(x[i])

In [14]:
#twopp
for i in range(500):
    tmax = x[i].argmax()
    tmin = x[i].argmin()
    twopp = tmax - tmin
    arr[i,7] = twopp

In [15]:
#pps
new_x = []
for i in range(500):
    ppt = np.ptp(x[i])
    tmax = x[i].argmax()
    tmin = x[i].argmin()
    twopp = tmax - tmin
    arr[i,8] = ppt/twopp
    new_x.append(ppt/twopp)

In [16]:
def get_rms_acceleration(signal, frame_size, hop_length):
    rms = []
    for i in range(0, len(signal), hop_length):
        current_rms = np.sqrt(np.sum(signal[i:i+frame_size]**2)/frame_size)
        rms.append(current_rms)
    return rms

In [17]:
new_x = []
for i in range(500):
    new_x.append(get_rms_acceleration(x[i], 1024,512))
 
    for j in range(9):
        arr[i,9+j] = new_x[i][j]

In [18]:
def get_margin_factor(signal, frame_size, hop_length):
    mar_fac = []
    for i in range(0, len(signal), hop_length):
        curr_mar_fac = np.max(np.abs(signal[i:i+frame_size])) / ((np.sum(np.sqrt(np.abs(signal[i:i+frame_size])))/ frame_size**2))
        mar_fac.append(curr_mar_fac)                             
    return mar_fac

In [19]:
new_x = []
for i in range(500):
    new_x.append(get_margin_factor(x[i], 1024,512))
    for j in range(9):
        arr[i,18+j] = new_x[i][j]

In [20]:
def get_shape_factor(signal, frame_size, hop_length):
    fin_shape_fact = []
    for i in range(0, len(signal), hop_length):
        cur_shape_fact = np.sqrt(((np.sum(signal[i:i+frame_size]**2))/frame_size) / (np.sum(np.abs(signal[i:i+frame_size]))/frame_size))
        fin_shape_fact.append(cur_shape_fact)

    return fin_shape_fact

In [21]:
new_x = []
for i in range(500):
    new_x.append(get_shape_factor(x[i], 1024,512))
    
    for j in range(9):
        arr[i,27+j] = new_x[i][j]

In [22]:
def get_impulse_factor(signal, frame_size, hop_length):
    impulse_factor = []
    for i in range(0, len(signal), hop_length):
        current_impls = max(np.abs(signal[i:i+frame_size]))/(np.sum(np.abs(signal[i:i+frame_size])/frame_size))
        impulse_factor.append(current_impls)
    return impulse_factor

In [23]:
new_x = []
for i in range(500):
    new_x.append(get_impulse_factor(x[i], 1024,512))
    
    for j in range(9):
        arr[i,36+j] = new_x[i][j]

In [24]:
def get_third_freq(signal, frame_size, hop_length):
    third = []
    for i in range(0, len(signal), hop_length):
        L = len(signal[i:i+frame_size])
        y = abs(np.fft.fft(signal[i:i+frame_size]/L))[:int(L/2)]
        current_third = (np.sum((y - (np.sum(y)/frame_size))**3))/(frame_size * (np.sqrt((np.sum((y - (np.sum(y)/frame_size))**2))/(frame_size-1)))**3)
        third.append(current_third)
    return np.array(third)

In [25]:
new_x = []
for i in range(500):
    new_x.append(get_third_freq(x[i], 1024,512))
    
    for j in range(8):
        arr[i,45+j] = new_x[i][j]

<ipython-input-24-1a8d04918b9b>:6: RuntimeWarning: invalid value encountered in double_scalars
  current_third = (np.sum((y - (np.sum(y)/frame_size))**3))/(frame_size * (np.sqrt((np.sum((y - (np.sum(y)/frame_size))**2))/(frame_size-1)))**3)


In [26]:
def get_forth_freq(signal, frame_size, hop_length):
    forth = []
    for i in range(0, len(signal), hop_length):
        L = len(signal[i:i+frame_size])
        y = abs(np.fft.fft(signal[i:i+frame_size]/L))[:int(L/2)]
        current_forth = (np.sum((y - (np.sum(y)/frame_size))**4))/(frame_size * ((np.sum((y - (np.sum(y)/frame_size))**2))/(frame_size-1))**2)
        forth.append(current_forth)
    return np.array(forth)

In [27]:
new_x = []
for i in range(500):
    new_x.append(get_forth_freq(x[i], 1024,512))
    
    for j in range(8):
        arr[i,53+j] = new_x[i][j]

<ipython-input-26-f472b2da5a18>:6: RuntimeWarning: invalid value encountered in double_scalars
  current_forth = (np.sum((y - (np.sum(y)/frame_size))**4))/(frame_size * ((np.sum((y - (np.sum(y)/frame_size))**2))/(frame_size-1))**2)


In [28]:
kf = KFold(n_splits=5, random_state = seed, shuffle = True)

In [29]:
kf.get_n_splits(arr)

5

In [30]:
for train_index, test_index in kf.split(arr):
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")

  Train: index=[  0   1   2   4   5   6  10  11  12  13  14  15  16  17  18  19  20  21
  22  23  24  26  27  28  29  30  31  32  33  34  35  36  39  40  42  44
  45  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63
  65  66  68  69  71  72  73  74  75  77  78  79  80  81  83  84  86  87
  88  89  90  91  92  93  94  95  97  98  99 101 102 104 105 106 107 108
 109 110 113 114 115 117 119 120 122 124 127 128 129 130 132 133 134 135
 137 138 139 140 141 142 143 144 145 146 148 149 150 151 152 153 154 155
 156 157 158 159 161 162 163 164 165 166 167 168 169 170 171 172 173 174
 175 176 177 178 181 182 183 184 185 186 187 188 189 190 191 192 196 198
 199 200 201 202 204 206 207 208 209 211 213 214 215 216 218 219 220 221
 223 224 225 226 227 228 229 230 233 234 235 236 237 238 239 240 241 243
 244 245 246 247 249 251 252 255 257 259 260 261 262 263 264 265 266 267
 268 269 270 271 272 273 274 275 276 277 278 280 281 282 284 285 286 287
 288 289 290 291 292 293 294 295 296

In [31]:
x_train, x_test, y_train, y_test = train_test_split(arr,y,random_state=seed,test_size=0.2)

In [32]:
print(arr.shape)
print(y.shape)
print(x_test.shape)

(500, 61)
(500, 1)
(100, 61)


In [33]:
clf2 = RandomForestClassifier(max_depth=2, random_state=0)
clf2.fit(x_train, y_train)
y_pred2 = clf2.predict(x_test)

<ipython-input-33-c2c638839e40>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf2.fit(x_train, y_train)


In [34]:
print(accuracy_score(y_test,y_pred2))
print(precision_score(y_test, y_pred2))
print(recall_score(y_test, y_pred2))

0.97
1.0
0.8846153846153846


In [35]:
clf3 = NearestCentroid()
clf3.fit(x_train, y_train)
y_pred3 = clf3.predict(x_test)

c:\users\lenovo\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [36]:
print(accuracy_score(y_test,y_pred3))
print(precision_score(y_test, y_pred3))
print(recall_score(y_test, y_pred3))

0.84
0.6923076923076923
0.6923076923076923
